<a href="https://colab.research.google.com/github/nneko1231/RoopColab/blob/main/DeepFake_V2.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Drive Mount

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = True  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

# 2. Installing

In [ ]:
!wget https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx -O inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx ./models

In [4]:
file_path = '/usr/local/lib/python3.11/dist-packages/jax/_src/dtypes.py'

with open(file_path, 'r') as f:
    file_contents = f.read()

file_contents = file_contents.replace('np.dtypes', 'np.dtype')

with open(file_path, 'w') as f:
    f.write(file_contents)

# 3. Settings

In [5]:
SOURCE_PATH = "SOURCE.jpg" #@param {type:"string"}
TARGET_PATH = "TARGET.mkv" #@param {type:"string"}
OUTPUT_PATH = "OUTPUT.mp4" #@param {type:"string"}
keep_fps = True  #@param {type:"boolean"}
keep_frames = True  #@param {type:"boolean"}
skip_audio = False  #@param {type:"boolean"}
temp_frame_format = "png" #@param ["jpg" , "png"]
#@markdown - Recommended 80–95 to balance quality and efficiency.
temp_frame_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
output_video_encoder = "h264_nvenc" #@param ["libx264","libx265","libvpx-vp9","h264_nvenc","hevc_nvenc"]
#@markdown - For professional or publication results, a value of 80+ is recommended.
#@markdown - For a quick test or preview, a value of 40–60 can be used.
output_video_quality = 80 # @param {"type":"slider","min":0,"max":100,"step":1}
max_memory = 12 # @param {"type":"slider","min":0.1,"max":12,"step":0.1}


# 4. Run

In [ ]:
def build_backend_command() -> str:
    command = (
        'python3 /content/roop/run.py' +
        f' -s "{SOURCE_PATH}" ' +
        f' -t "{TARGET_PATH}" ' +
        f' -o "{OUTPUT_PATH}" ' +
        f' --execution-provider cuda ' +  # Always use CUDA for GPU acceleration
        f' --frame-processor face_swapper face_enhancer '  # Always use face_swapper and face_enhancer
        f' --execution-threads 8'
    )

    # Add `--keep-fps` only if keep_fps is True
    if keep_fps:
        command += ' --keep-fps '

    # Add `--keep-frames` only if keep_frames is True
    if keep_frames:
        command += ' --keep-frames '

    # Add `--skip-audio` only if skip_audio is True
    if skip_audio:
        command += ' --skip-audio '

    # Add the rest of the parameters
    command += (
        f' --temp-frame-format {temp_frame_format} ' +  # Temp frame format (jpg or png)
        f' --temp-frame-quality {temp_frame_quality} ' +  # Temp frame quality (0-100)
        f' --output-video-encoder {output_video_encoder} ' +  # Output video encoder (e.g., h264_nvenc)
        f' --output-video-quality {output_video_quality} ' +  # Output video quality (0-100)
        f' --max-memory {max_memory} '  # Max memory usage in GB
    )
    return command

# Write the command to a shell script
with open("command.sh", "w") as f:
    f.write(build_backend_command())

# Run the command from the shell script
!bash command.sh

# Remove the command.sh file after execution
import os
os.remove("command.sh")
